# Points

Data Source - https://msi.nga.mil/Publications/WPI

- Convert the excel file into a csv and save to the resources folder. 
- Alternatively, there is a ports.csv file in the resources folder with a series of lat/longs placed randomly at the coast. 

In [ ]:
import pandas as pd
ports = pd.read_csv("../resources/ports.csv")

In [ ]:
print(ports)

In [ ]:
from shapely.geometry import Point
import geopandas as gpd

port_geometry = [Point(xy) for xy in zip(ports['lon'], ports['lat'])]
port_geodata = gpd.GeoDataFrame(ports, crs="EPSG:4326", geometry=port_geometry)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
port_geodata.plot(ax=ax)
plt.show()

In [ ]:
import cartopy.crs as ccrs

fig, ax = plt.subplots(subplot_kw={'projection': ccrs.Robinson()})
port_geodata.plot(ax=ax, transform=ccrs.PlateCarree())
plt.show()

#### Exercise


My attempt

![alt text](../resources/assets/ports.png "Title")


## Plotting Lines

Data Source - https://openflights.org/data.html

- Download airports.dat and place in the resources folder
- Download routes.dat and place in the resources folder

In [ ]:
airports = pd.read_csv("../resources/airports", delimiter=',', names=['id', 'name', 'city', 'country', 'iata', 
                                                                   'icao', 'lat', 'long', 'altitude', 'timezone',
                                                                   'dst', 'tz', 'type', 'source'])

In [ ]:
print(airports)

### Exercise

- Create a list of points from the lat/long values in the airports DataFrame
- Convert the airports dataframe to a GeoDataFrame using the list of points created above
- Generate a map of the worlds airports

In [ ]:
routes = pd.read_csv("../resources/routes", delimiter=',', names=['airline', 'id', 'source_airport', 'source_airport_id',
                                                               'destination_airport', 'destination_airport_id', 'codeshare',
                                                               'stops', 'equitment'])

In [ ]:
print(routes)

In [ ]:
source_airports = airports[['name', 'iata', 'icao', 'lat', 'long']]
destination_airports = source_airports.copy()
source_airports.columns = [str(col) + '_source' for col in source_airports.columns]
destination_airports.columns = [str(col) + '_destination' for col in destination_airports.columns]

In [ ]:
routes = routes[['source_airport', 'destination_airport']]
routes = pd.merge(routes, source_airports, left_on='source_airport', right_on='iata_source')
routes = pd.merge(routes, destination_airports, left_on='destination_airport', right_on='iata_destination')

print(routes.columns)

In [ ]:
from shapely.geometry import LineString

routes_geometry = [LineString([[routes.iloc[i]['long_source'], routes.iloc[i]['lat_source']], [routes.iloc[i]['long_destination'], routes.iloc[i]['lat_destination']]]) for i in range(routes.shape[0])]
routes_geodata = gpd.GeoDataFrame(routes, geometry=routes_geometry, crs='EPSG:4326')

In [ ]:
print(routes.columns)

In [ ]:
fig, ax = plt.subplots(figsize=(20,20))
ax.patch.set_facecolor('black')

routes_geodata.plot(ax=ax, color='white', linewidth=0.1)

plt.show()

In [ ]:
fig, ax = plt.subplots(subplot_kw={'projection': ccrs.Robinson()}, figsize=(20,20))
ax.patch.set_facecolor('black')
routes_geodata.plot(ax=ax, transform=ccrs.Geodetic(), color='white', linewidth=0.1, alpha=0.1)
plt.setp(ax.spines.values(), color='black')
plt.setp([ax.get_xticklines(), ax.get_yticklines()], color='black')
ax.set_ylim(-7000000, 8800000)
plt.show()

### Exercise

- Use the routes data to add a new column to the airports DataFrame with a value showing the number of flights that landed in that airport
- Create a plot combining the routes and airports into one
- Scale the size of the airport points according to the number of flights that land there.
- The expected output is shown below.

![alt text](../resources/assets/airroutes_mad.png "Title")